# Rocket performance evaluation

The purpose of this notebook is to provide some initial evaluation of rocket performance.
This particular evaluation is for two stages going to LEO.

A module of naming conventions and conversion constants is used to provide convenient naming, etc. An underscore is appended on names to minimize collision with variable names.  For example, initial and final mass are "m0_" and "m1_".

Also, import Python standard math module.

In [1]:
from conventions import *    # includes unit conversions and useful constant names.
import math

## Launch objective

The objective is to get a payload of about 55 lb-m to LEO. The cumulative delta V requirement to get to LEO is set at 11.0 km/sec.

For convenience, we specify both initial and final mass for the payload, although it does not hcange.

In [2]:
target_dV = 11000.0    # m/s
payload = {
    name_ : 'satellite',
    m0_ : 55.1 * lb2kg,  # payload
    MR_ : 1.0,
}
payload[m1_] = payload[m0_]    # payload does not change mass

## Vehicle

Below is a basic description of the rocket.  Notation is Python variables, often in "dictionaries".

In [3]:
stage1 = {
    name_ : 'CSTR 4A',
    m0_ : 25737.0 * lb2kg,
    m1_ : 3239.0 * lb2kg,
    isp_ : 265.3, # sec
    Tpeak_ : 120880.0, # lbf
    Tavg_  : 108190.0, # lbf
}
stage2 = {
    name_ : 'BARK',  # Build A Rocket Kit
    m0_ : 8500.0 * lb2kg,
    m1_ : 800.0 * lb2kg,
    isp_ : 340.0, # sec
}

### Mass ratios - individual stages

Mass ratio = initial mass / final mass.  That is, MR > 1.0.

In [4]:
headings = {
    'name': 'stage name',
    'm0': 'm0(kg)',
    'm1': 'm1(kg)',
    'MR': 'MR'}
print('{name:<16} {m0:^11} {m1:^11} {MR:^11}'.format(**headings))

for stage in (stage1, stage2, payload):
    stage[MR_] = stage[m0_] / stage[m1_]
    print('{name:<16s} {m0:11.4f} {m1:11.4f} {MR:11.5f}'.format(**stage))

stage name         m0(kg)      m1(kg)        MR     
CSTR 4A           11674.1068   1469.1857     7.94597
BARK               3855.5351    362.8739    10.62500
satellite            24.9929     24.9929     1.00000


### Mass ratios - Vehicle

In [5]:
# vehicle cumulative
upper = stage2[m0_] + payload[m0_]
stage1V = {
    name_ : 'stages 1 + 2',
    m0_ : stage1[m0_] + upper,
    m1_ : stage1[m1_] + upper,
}
upper = payload[m0_]
stage2V = {
    name_ : 'stage 2',
    m0_ : stage2[m0_] + upper,
    m1_ : stage2[m1_] + upper,
}

print('{name:<16} {m0:^11} {m1:^11} {MR:^11}'.format(**headings))
for stage in (stage1V, stage2V):
    stage[MR_] = stage[m0_] / stage[m1_]
    print('{name:<16s} {m0:11.4f} {m1:11.4f} {MR:11.5f}'.format(**stage))

stage name         m0(kg)      m1(kg)        MR     
stages 1 + 2      15554.6349   5349.7138     2.90756
stage 2            3880.5281    387.8668    10.00479


## delta V desired

Compute stage 1 deltaV requirement for LEO.

In [6]:
for stage in (stage1, stage2):
    stage[dVdsrd_] = (stage[isp_] / (stage1[isp_]+stage2[isp_])) * target_dV
    print("{name:<16} dV {dVdsrd:11.4f}".format(**stage))
print("dV sum {:24.4f}".format(sum([stage[dVdsrd_] for stage in (stage1, stage2)])))

CSTR 4A          dV   4821.2457
BARK             dV   6178.7543
dV sum               11000.0000


## delta V expected

In [7]:
stage1V[isp_] = stage1[isp_]
stage2V[isp_] = stage2[isp_]
for stage in (stage1V, stage2V):
    stage[dV_] = stage[isp_]*gEarth*math.log(stage[MR_])
    print("{name:<16} dV {dV:11.4f}".format(**stage))
print("dV sum {:24.4f}".format(sum([stage[dV_] for stage in (stage1V, stage2V)])))

stages 1 + 2     dV   2776.8395
stage 2          dV   7679.0180
dV sum               10455.8575


## Stage 1 acceleration

In [13]:
maxG = stage1[Tavg_] / (stage1V[m1_]*kg2lb)  # final G is avg thrust
accelmaxG = maxG * gEarth
initG = stage1[Tpeak_] / (stage1V[m0_]*kg2lb)
print("max G", maxG)
print("max accel (m/s/s)", accelmaxG)
print("init G", initG)

max G 9.17323068313818
max accel (m/s/s) 89.95866267879703
init G 3.525010133529297
